In [ ]:
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split
import numpy as np

(train_input,train_target),(test_input,test_target) = imdb.load_data(num_words=500)
print(train_input.shape,train_target.shape)
print(len(train_input[0]))
print((train_input[0]))
#긍정과 부정으로 이진분류
print(train_target[:20])

In [ ]:
train_input, val_input, train_target, val_target = train_test_split(train_input,train_target,test_size=0.2,random_state=42)
#array에 넣음으로써 한 세트의 길이를 파악하기 용이하게 변경
lengths = np.array([len(x) for x in train_input])
print(np.mean(lengths),np.median(lengths),max(lengths))
#문장길이를 통일시킨다
from tensorflow.keras.preprocessing.sequence import pad_sequences
#문장의 길이를 100으로 한정하여 이제 2차원 배열로 맞춰졌다
train_seq = pad_sequences(train_input,maxlen=100)
val_seq = pad_sequences(val_input,maxlen=100)



In [ ]:
#한 토큰을 원핫인코딩으로 변경함
train_oh = keras.utils.to_categorical(train_seq)
val_oh = keras.utils.to_categorical(val_seq)
#모델생성
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(8,input_shape=(100,500)))
model.add(keras.layers.Dense(1,activation='sigmoid'))
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best_simplernn_model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
history = model.fit(train_oh,train_target,epochs=100,batch_size=64,validation_data=(val_oh,val_target),callbacks=[checkpoint_cb,early_stopping_cb])
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','test'])
plt.show()

In [ ]:
#벡터임베딩 방법으로 진행하는것이 메모리 효율적이다
#모델생성
model2 = keras.Sequential()
model2.add(keras.layers.Embedding(500,16,input_shape=100))
model2.add(keras.layers.SimpleRNN(8))
model2.add(keras.layers.Dense(1,activation='sigmoid'))
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model2.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best_embedding_model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
history = model2.fit(train_seq,train_target,epochs=100,batch_size=64,validation_data=(val_seq,val_target),callbacks=[checkpoint_cb,early_stopping_cb])
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','test'])
plt.show()